In [1]:
import os
from pathlib import Path
import json
from dotenv import load_dotenv
from pinecone import Pinecone,ServerlessSpec
from pypdf import PdfReader
from langchain.text_splitter import(
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter
)
import re


In [7]:
BASE_DIR = Path(os.getcwd()).parent.parent

In [8]:
%run ../paths.ipynb
%run ./cleaner_functions.ipynb

download paths
cleaner functions executed


In [11]:
tokens_file= open(TOKENS_THESIS,"r",encoding="utf-8")
tokens_json = json.load(tokens_file)

In [12]:
reader_thesis = PdfReader(FILE_THESIS)

In [18]:
def tokenizer_1(reader_pdf: PdfReader,token_json:dict,description:str) -> dict:
    pdf_texts = [p.extract_text().strip() for p in reader_pdf.pages]
    character_splitter = RecursiveCharacterTextSplitter(
        separators = ["\n\n","\n",". ", " ", ""],
        chunk_size = 1000,
        chunk_overlap=0
    )

    character_split_texts = character_splitter.split_text(
        "\n\n".join(pdf_texts)
    )

    token_json["first_tokenization"] = { 
        "values" : character_split_texts, 
        "description":description
    }

    token_json['first_tokenization']['values'] = [strip_control_chars(text)  for text in token_json['first_tokenization']['values']]
    return token_json


tokens = tokenizer_1(
    reader_pdf=reader_thesis,
    token_json=tokens_json,
    description="tokens with recursive tokenization, without cleaning or another technique"
)



In [19]:
file = open(TOKENS_THESIS,'w',encoding="utf-8")
json.dump(tokens,file,indent=4)
